In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import glob
from matplotlib.colors import Normalize

In [2]:
# Pasta onde estão os arquivos CSV
folder = "/mnt/data"
csv_files = glob.glob(os.path.join(folder, "scan_*_from_*_to_*.csv"))

In [3]:
# Inicializa dicionário para armazenar os dados
data_dict = {}

# Extrai os dados e organiza por (from_slot, to_slot)
for file in csv_files:
    # Extrai slots do nome do arquivo
    match = re.search(r"scan_.*?_from_(.*?)_to_(.*?)_", os.path.basename(file))
    if match:
        from_slot = match.group(1)
        to_slot = match.group(2)
    else:
        continue  # ignora se não casar

    # Lê o conteúdo, pulando cabeçalho até encontrar "2d statistical"
    with open(file, 'r') as f:
        lines = f.readlines()
    start = next(i for i, line in enumerate(lines) if line.startswith("2d statistical"))
    df = pd.read_csv(file, skiprows=start + 1, index_col=0)
    df.columns = df.columns.astype(float)
    df.index = df.index.astype(float)
    data_dict[(from_slot, to_slot)] = df